1. generar archivo HTML
2. renderizar archivo a png
3. recortar archivo y guardarlo

In [1]:
import imgkit
from jinja2 import Environment, FileSystemLoader, select_autoescape
import json
from skimage import io
import numpy as np
from os import listdir
from fpdf import FPDF
import math
import glob
import os

In [32]:
env = Environment(
    loader=FileSystemLoader("./"),
    autoescape=select_autoescape()
)
template = env.get_template("./templates/front.html")

data = json.load(open('info.json',))

def crop_card(img):
    white = np.array([255, 255, 255, 255])
    mask = np.abs(img - white).sum(axis=2) < 0.05

    # Find the bounding box of those pixels
    coords = np.array(np.nonzero(~mask))
    top_left = np.min(coords, axis=1)
    bottom_right = np.max(coords, axis=1)

    margin = 1
    out = img[top_left[0]:bottom_right[0],
                top_left[1]:bottom_right[1]]
    new_shape = tuple(map(sum, zip(out.shape,(margin*2,margin*2,0))))
    new_image = np.full(new_shape,255,dtype=np.uint8)
    new_image[margin:-margin,margin:-margin] = out
    return new_image

In [33]:
options = {
    #"javascript-delay": 10000,
    "enable-local-file-access": None,
    "debug-javascript": None,
}

# Delete old images
files = glob.glob('./png_out/*')
for f in files:
    os.remove(f)

# Generate new images
for card in data['cards']:
    html_path = './html_out/'+card['name']+'.html'
    png_path = './png_out/'+card['name']+'_1.png'

    template.stream(card).dump(html_path) # Create HTML

    imgkit.from_file(html_path, png_path, options=options) # Convert HTML to PNG
    image = io.imread(png_path)
    image = crop_card(image) # Crop image

    for i in range(card['amount']): # Multiply number of images
        png_path = './png_out/{}_{}.png'.format(card['name'],i+1)
        io.imsave(png_path,image)

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


In [4]:
# Generate PDF

cards_path_list = listdir('./png_out')
cards_path_list.sort()
num_pages = math.ceil(len(cards_path_list)/9)
width = 2.5
height = 3.5
margin = 0.4
pdf = FPDF('P', 'in', 'A4')

for index, card_path in enumerate(cards_path_list):
    if index % 9 == 0: pdf.add_page()
    pos = index % 9
    x_pos = margin + width*(pos%3)
    y_pos = margin + height*math.floor(pos/3)
    pdf.image('png_out/'+cards_path_list[index], x = x_pos, y = y_pos, w=2.5)

pdf.output('./pdf_out/test.pdf', 'F')

''